# 1. Config

## 1.1 Training Configs

In [ ]:
BASE_PATH = "/content/drive/MyDrive/Couplet-Data/final/20250711-MiscData/"
DATA_FORMAT = "zh-vi"
CHECKPOINT_DIR = "checkpoints"
VENV_PATH = "venv"
MODEL_PATH = f"{CHECKPOINT_DIR}/model.pt"
BPE_CODES_PATH = f"{CHECKPOINT_DIR}/bpecodes"

## 1.2 Evaluation Configs

# 2. Setup
Install dependencies, create and activate virtual environment, and verify installations.

In [ ]:
# =====================
# 2. Setup (Environment & Dependencies)
# =====================
# Install system dependencies (if needed)
!sudo apt update
!sudo apt install python3.10 python3.10-venv python3.10-dev

# Create virtual environment
!python3.10 -m venv venv

# Activate virtual environment
!source venv/bin/activate

# Upgrade pip
!venv/bin/python -m pip install --upgrade pip==23

# Check Python and pip version
!venv/bin/python --version
!venv/bin/python -m pip --version

# Install required Python packages
!venv/bin/python -m pip install numpy==1.23.3 --force-reinstall
!venv/bin/python -m pip install fairseq
!venv/bin/python -m pip install "torch==1.11.0+cu113" --index https://download.pytorch.org/whl/
!venv/bin/python -m pip install cython sacremoses sentencepiece
!venv/bin/python -m pip install tensorboardX
!venv/bin/python -m pip install fastBPE

In [ ]:
# Download pretrained model and extract
!mkdir -p checkpoints
!wget https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz
!tar -xzvf wmt19.en-de.joined-dict.single_model.tar.gz -C checkpoints

In [ ]:
# Move model files to checkpoints
!mv /content/checkpoints/wmt19.en-de.joined-dict.single_model/dict.de.txt checkpoints/
!mv /content/checkpoints/wmt19.en-de.joined-dict.single_model/dict.en.txt checkpoints/
!mv /content/checkpoints/wmt19.en-de.joined-dict.single_model/model.pt checkpoints/
!mv /content/checkpoints/wmt19.en-de.joined-dict.single_model/bpecodes checkpoints/

In [ ]:
# Example: Interactive translation
!echo "Hello, how are you?" | \
    venv/bin/fairseq-interactive checkpoints \
    --path checkpoints/model.pt \
    --source-lang en --target-lang de \
    --tokenizer moses --bpe fastbpe --bpe-codes checkpoints/bpecodes \
    --beam 5 --remove-bpe

# 3. Data Preprocessing

## FastBPE setup and usage

In [ ]:
# Clone and build FastBPE
!git clone https://github.com/glample/fastBPE.git
!cd fastBPE && /content/venv/bin/python setup.py install
!cd fastBPE && g++ -std=c++11 -pthread -O3 fastBPE/main.cc -o fastbpe
!mv fastBPE/fastbpe /content/venv/bin/

In [ ]:
# Change directory to fastBPE
!cd fastBPE

In [ ]:
# Build and install FastBPE
!cd fastBPE && /content/venv/bin/python setup.py install
!cd fastBPE && g++ -std=c++11 -pthread -O3 fastBPE/main.cc -o fastbpe
!mv fastBPE/fastbpe /content/venv/bin/

In [ ]:
# Set base path for data
BASE_PATH = "/content/drive/MyDrive/Couplet-Data/final/20250711-MiscData/"

In [ ]:
# Apply BPE to training, validation, and test data
for split in ["train", "valid", "test"]:
    for lang in ["zh", "vi"]:
        input_file = f"{BASE_PATH}{split}.{lang}"
        output_file = f"{BASE_PATH}{split}.{lang}.bpe"
        !venv/bin/fastbpe applybpe "{output_file}" "{input_file}" "/content/checkpoints/bpecodes"

In [ ]:
# Count lines in original and BPE-processed files
def count_lines(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return len(f.readlines())

original_lines = count_lines(f"{BASE_PATH}train.zh")
bpe_lines = count_lines(f"{BASE_PATH}train.zh.bpe")
print(f"Original lines: {original_lines}, BPE lines: {bpe_lines}")

In [ ]:
# Check line counts for BPE files
for split in ["train", "valid", "test"]:
    for lang in ["zh", "vi"]:
        bpe_file = f"{BASE_PATH}{split}.{lang}.bpe"
        !wc -l "{bpe_file}"

In [ ]:
# Rename BPE files for Fairseq
for split in ["train", "valid", "test"]:
    for lang in ["zh", "vi"]:
        src = f"{BASE_PATH}{split}.{lang}.bpe"
        dst = f"{BASE_PATH}{split}.zh-vi.{lang}"
        !mv "{src}" "{dst}"

In [ ]:
# Preprocess data for Fairseq
!/content/venv/bin/fairseq-preprocess \
    --source-lang zh --target-lang vi \
    --trainpref "{BASE_PATH}train.zh-vi" \
    --validpref "{BASE_PATH}valid.zh-vi" \
    --testpref "{BASE_PATH}test.zh-vi" \
    --destdir data-bin/ \
    --joined-dictionary \
    --workers 4

In [ ]:
!wc -l "{base_path}train.zh-vi.zh" "{base_path}train.zh-vi.vi"
!wc -l "{base_path}valid.zh-vi.zh" "{base_path}valid.zh-vi.vi"
!wc -l "{base_path}test.zh-vi.zh" "{base_path}test.zh-vi.vi"


In [ ]:
!cat "{base_path}test.zh-vi.zh"


# 4. Training

In [ ]:
# Train Fairseq transformer model
!/content/venv/bin/fairseq-train data-bin/ \
    --arch transformer_wmt_en_de_big \
    --share-all-embeddings \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --max-source-positions 4096 --max-target-positions 4096 \
    --skip-invalid-size-inputs-valid-test \
    --update-freq 16 \
    --save-interval 2 \
    --keep-best-checkpoints 1 \
    --keep-last-epochs 2 \
    --save-dir /content/drive/MyDrive/Couplet-Data/FairSeq/checkpoints/ \
    --restore-file /content/drive/MyDrive/Couplet-Data/FairSeq/checkpoints/checkpoint_last.pt \
    --fp16 \
    --max-epoch 30 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --patience 5

# 5. Inference

In [ ]:
# Example: Inference on a Chinese sentence
!echo "不厭湖上月 [SEP] 宛在水中央" | \
    venv/bin/fairseq-interactive data-bin \
    --path /content/drive/MyDrive/Couplet-Data/FairSeq/checkpoints/checkpoint.best_bleu_22.8800.pt \
    --source-lang zh --target-lang vi \
    --tokenizer moses --bpe fastbpe --bpe-codes checkpoints/bpecodes \
    --beam 5 --remove-bpe

# 6. Evaluation

In [ ]:
!/content/venv/bin/fairseq-generate data-bin/ \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 32 --beam 3 --remove-bpe

In [ ]:
# Batch inference for multiple sentences
import subprocess

sentences = [
    "潑島墨痕湖水滿 [SEP] 擎天筆勢石峰高",
    "不厭湖上月 [SEP] 宛在水中央",
    "平湖長日月 [SEP] 故甸小乾坤",
    "高低韻送橫江鳥 [SEP] 紅紫頻開隔岸花",
    "孤山木蔭三千界 [SEP] 勺水波澄九十春",
    "夜月或過仙是鶴 [SEP] 濠梁信樂子非魚",
    "惟聖不知惟神不測 [SEP] 在天成象在地成形",
    "道有主張斗北文明之象 [SEP] 人同瞻仰交南禮樂之都",
    "竇桂王槐國家楨幹 [SEP] 唐科宋榜士子梯階",
    "火宅蓮開今古夢 [SEP] 金門鐘響草花寒",
]

base_command = [
    "venv/bin/fairseq-interactive", "data-bin",
    "--path", "/content/drive/MyDrive/Couplet-Data/FairSeq/backup/fairseq_checkpoint_best.pt",
    "--source-lang", "zh", "--target-lang", "vi",
    "--tokenizer", "moses", "--bpe", "fastbpe",
    "--bpe-codes", "checkpoints/bpecodes",
    "--beam", "5", "--remove-bpe"
    ]

for sentence in sentences:
    print(f"\n>>> Input: {sentence}")
    process = subprocess.Popen(
        base_command,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    stdout, stderr = process.communicate(input=sentence)
    print(stdout)
    if stderr:
        print(f"[stderr]: {stderr}")